# Analysis Folder Setup

In [33]:
import os, glob
from IPython.core import display as ICD
import pandas as pd
import subprocess
from shutil import copy2

In [34]:
### Setting up onsets folders

In [35]:
subject_ids = sorted([x.split("/")[-1] for x in 
                      glob.glob("/projects/niblab/bids_projects/Experiments/bbx/bids/sub-*")])
print(subject_ids)


['sub-001', 'sub-002', 'sub-003', 'sub-004', 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-009', 'sub-010', 'sub-011', 'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 'sub-017', 'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 'sub-028', 'sub-030', 'sub-031', 'sub-032', 'sub-033', 'sub-034', 'sub-035', 'sub-036', 'sub-037', 'sub-038', 'sub-039', 'sub-040', 'sub-041', 'sub-042', 'sub-043', 'sub-044', 'sub-045', 'sub-046', 'sub-048', 'sub-050', 'sub-051', 'sub-052', 'sub-053', 'sub-054', 'sub-055', 'sub-056', 'sub-057', 'sub-058', 'sub-059', 'sub-060', 'sub-061', 'sub-062', 'sub-063', 'sub-064', 'sub-065', 'sub-066', 'sub-067', 'sub-068', 'sub-070', 'sub-071', 'sub-072', 'sub-073', 'sub-074', 'sub-075', 'sub-076', 'sub-077', 'sub-078', 'sub-079', 'sub-080', 'sub-082', 'sub-083', 'sub-084', 'sub-085', 'sub-086', 'sub-087', 'sub-088', 'sub-089', 'sub-090', 'sub-091', 'sub-092', 'sub-093', 'sub-095', 'sub-096', 'sub-097'

In [5]:
deriv_path = "/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives"

In [5]:
for sub_id in subject_ids:
    
    # set and make directory paths
    subject_folder=os.path.join(deriv_path, sub_id, "ses-1")
    
    if os.path.exists(subject_folder):
        pass
    else:
        os.makedirs(subject_folder)
        
    anat_path = os.path.join(subject_folder, "anat")
    if os.path.exists(anat_path):
        pass
    else:
        os.makedirs(anat_path)
        
    func_path = os.path.join(subject_folder, "func")
    if os.path.exists(func_path):
        pass
    else:
        os.makedirs(func_path)
        
    onset_path = os.path.join(subject_folder, "func/onsets")
    if os.path.exists(onset_path):
        pass
    else:
        os.makedirs(onset_path)
        
    motion_path = os.path.join(subject_folder, "func/motion_assessment")
    if os.path.exists(motion_path):
        pass
    else:
        motion_param_path = os.path.join(motion_path, "motion_parameters")
        os.makedirs(motion_param_path)
        os.makedirs(motion_path)
    
    #print("Anatomical path: \t{} \nFunctional path: \t{} \nOnset path: \t{} \nMotion path: \t{}".format(anat_path, 
                                                                                                       #func_path,
                                                                                                       #onset_path,
                                                                                                       #motion_path))
    move_onsets = False          
                
    # move onsets to subject folders
   
    if move_onsets == True:
        orig_onsets=glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bbx/clean_onsets", "{}_*".format(sub_id)))

        for onset in orig_onsets:
            print("Onset path: \t{} \nOnset: {} \n\n".format(onset_path, onset))
            copy2(onset, onset_path)
            
    # generate visual report
    
    copy_imgs = False
    
    if copy_imgs == True:
        # get target images
        img_path="/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep_images"
        fmriprep_imgs = glob.glob(os.path.join(img_path, "{}_*".format(sub_id)))
        if not fmriprep_imgs:
            anat_imgs=glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/{}/figures/{}_*T1w.svg".format(sub_id, sub_id)))
            mask_imgs=glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/{}/ses-1/figures/{}_*-brain_mask.svg".format(sub_id, sub_id)))
            sdc_imgs=glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/{}/ses-1/figures/{}_*-sdc_bold.svg".format(sub_id, sub_id)))
            fmap_imgs=glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/{}/ses-1/figures/{}_*-fieldmap_bold.svg".format(sub_id, sub_id)))
            flirt_imgs=glob.glob(os.path.join("/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/{}/ses-1/figures/{}_*-flirtbbr_bold.svg".format(sub_id, sub_id)))
        
            for grp in [anat_imgs, mask_imgs, sdc_imgs, fmap_imgs, flirt_imgs]:
                for img in grp:
                    print("Copying {} to {} \n".format(img, img_path))
                    copy2(img, img_path)

In [6]:
### Setup Confounds

outlier_cmd = "fsl_motion_outliers -i %s  -o %s --fd --thresh=%s -p %s -v > %s"%(filename, confound_path, arglist["MOCO"], plot_path, outlier_path)


In [ ]:
for sub_id in subject_ids:
    print("Subject: \t{}".format(sub_id))
    
    funcs=glob.glob(os.path.join(deriv_path, "{}/ses-1/func/*brain.nii.gz".format(sub_id)))
    for func in sorted(funcs):
        print(func)
        task = func.split("/")[-1].split("_")[2]
        run = func.split("/")[-1].split("_")[3]
        
        if task != "resting":
            confound_path =os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/{}_ses-1_{}_{}_confound.txt".format(sub_id,sub_id, task, run))
            outlier_path = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/{}_ses-1_{}_{}_outlier_output.txt".format(sub_id,sub_id, task, run))
        else:
            confound_path =os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/{}_ses-1_{}".format(sub_id,sub_id, task))
            outlier_path = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/{}_ses-1_{}".format(sub_id,sub_id, task))

        outlier_cmd = "fsl_motion_outliers -i %s  -o %s --fd --thresh=0.9 -v > %s"%(func, confound_path, outlier_path)
        print(">>> {} \n".format(outlier_cmd))
        os.system(outlier_cmd)#, shell=True)

Subject: 	sub-001
/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-001/ses-1/func/sub-001_ses-1_task-resting_space-MNI152NLin2009cAsym_desc-preproc_bold_brain.nii.gz
>>> fsl_motion_outliers -i /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-001/ses-1/func/sub-001_ses-1_task-resting_space-MNI152NLin2009cAsym_desc-preproc_bold_brain.nii.gz  -o /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-001/ses-1/func/motion_assessment/sub-001_ses-1_task-resting_space-MNI152NLin2009cAsym_confound.txt --fd --thresh=0.9 -v > /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-001/ses-1/func/motion_assessment/sub-001_ses-1_task-resting_space-MNI152NLin2009cAsym_outlier_output.txt 

/projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-001/ses-1/func/sub-001_ses-1_task-training_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold_brain.nii.gz
>>> fsl_motion_outliers -i /projects/niblab/bids_projects/Experiments/bbx/bids

In [26]:
for sub_id in subject_ids:
    
    try:
        #print("Subject: \t{}".format(sub_id))
        tsvs = glob.glob(os.path.join(deriv_path, "fmriprep/{}/ses-1/func/{}_ses-1_task-*-confounds_regressors.tsv".format(sub_id, sub_id)))
        for tsv in tsvs:
            task=tsv.split("/")[-1].split("_")[2]
            run=tsv.split("/")[-1].split("_")[3]
            
            
            if "resting" not in task:
                moco0 = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/motion_parameters/{}_ses-1_{}_{}_moco0.txt".format(sub_id,sub_id, task, run))
                moco1 = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/motion_parameters/{}_ses-1_{}_{}_moco1.txt".format(sub_id,sub_id, task, run))
                moco2 = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/motion_parameters/{}_ses-1_{}_{}_moco2.txt".format(sub_id,sub_id, task, run))
                moco3 = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/motion_parameters/{}_ses-1_{}_{}_moco3.txt".format(sub_id,sub_id, task, run))
                moco4 = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/motion_parameters/{}_ses-1_{}_{}_moco4.txt".format(sub_id,sub_id, task, run))
                moco5 = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/motion_parameters/{}_ses-1_{}_{}_moco5.txt".format(sub_id,sub_id, task, run))
                
                confound_path =os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/{}_ses-1_{}_{}_confound.txt".format(sub_id,sub_id, task, run))
                outlier_path = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/{}_ses-1_{}_{}_outlier_output.txt".format(sub_id,sub_id, task, run))
            else:
                moco0 = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/motion_parameters/{}_ses-1_{}_moco0.txt".format(sub_id,sub_id, task))
                moco1 = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/motion_parameters/{}_ses-1_{}_moco1.txt".format(sub_id,sub_id, task))
                moco2 = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/motion_parameters/{}_ses-1_{}_moco2.txt".format(sub_id,sub_id, task))
                moco3 = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/motion_parameters/{}_ses-1_{}_moco3.txt".format(sub_id,sub_id, task))
                moco4 = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/motion_parameters/{}_ses-1_{}_moco4.txt".format(sub_id,sub_id, task))
                moco5 = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/motion_parameters/{}_ses-1_{}_moco5.txt".format(sub_id,sub_id, task))

                confound_path =os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/{}_ses-1_{}_confound.txt".format(sub_id,sub_id, task))
                outlier_path = os.path.join(deriv_path, "{}/ses-1/func/motion_assessment/{}_ses-1_{}".format(sub_id,sub_id, task))


            
            
            """df = pd.read_csv(tsv, sep="\t")
            #print(df.columns.values)
            df['trans_x'].to_csv(moco0, index=False)
            df['trans_y'].to_csv(moco1, index=False)
            df['trans_z'].to_csv(moco2, index=False)
            df['rot_x'].to_csv(moco3, index=False)
            df['rot_y'].to_csv(moco4, index=False)
            df['rot_z'].to_csv(moco5, index=False)

            #'trans_y','trans_z','rot_x', 'rot_y', 'rot_z']]
            #ICD.display(mocos)
            #confound = df[["motion_outlier00", "motion_outlier01", "motion_outlier02", "motion_outlier03", "motion_outlier04", "motion_outlier05"]]
            #ICD.display(confound)
            #print(confound_path)
            #confound.to_csv(confound_path, index=False, sep="\t")
            #temp_df = confound.apply(pd.value_counts).fillna(0)
            #ICD.display(temp_df)
            #print(temp_df.iloc[1,].sum())"""
    except:
        print("BAD SUBJECT: ", sub_id)
        """print("Found confound, checking for outliers")
        for i in range(0,6):
            x = confound['motion_outlier0{}'.format(i)].loc[confound["motion_outlier0{}".format(i)] == 1].any()
            if x == True:
                pass #print(">>> Column {} being written".format(confound['motion_outlier0{}'.format(i)]))
            else:
                print("no 1s found!")"""

/home/nbytes/modules/software/miniconda3/3.0.0/lib/python3.7/site-packages/ipykernel_launcher.py:36: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
/home/nbytes/modules/software/miniconda3/3.0.0/lib/python3.7/site-packages/ipykernel_launcher.py:37: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
/home/nbytes/modules/software/miniconda3/3.0.0/lib/python3.7/site-packages/ipykernel_launcher.py:38: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
/home/nbytes/modules/software/miniconda3

In [38]:
for sub_id in subject_ids:
    t1_file = os.path.join(deriv_path, "fmriprep/{}/anat/{}_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz".format(sub_id,sub_id))
    anat_path = os.path.join(deriv_path, "{}/ses-1/anat".format(sub_id))
    
    
    try:
        print("COPYING {} to {} \n\n".format(t1_file, anat_path))
        copy2(t1_file, anat_path)
    except:
        print("bad subject ", sub_id)

COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-001/anat/sub-001_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-001/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-002/anat/sub-002_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-002/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-003/anat/sub-003_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-003/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-004/anat/sub-004_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-004/ses-1/anat 


COPYING /projects/niblab/bids_projec

COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-037/anat/sub-037_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-037/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-038/anat/sub-038_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-038/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-039/anat/sub-039_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-039/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-040/anat/sub-040_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-040/ses-1/anat 


COPYING /projects/niblab/bids_projec

COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-074/anat/sub-074_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-074/ses-1/anat 


bad subject  sub-074
COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-075/anat/sub-075_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-075/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-076/anat/sub-076_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-076/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-077/anat/sub-077_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-077/ses-1/anat 


COPYING /projec

COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-115/anat/sub-115_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-115/ses-1/anat 


bad subject  sub-115
COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-116/anat/sub-116_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-116/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-117/anat/sub-117_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-117/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-118/anat/sub-118_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-118/ses-1/anat 


bad subject  su

COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-154/anat/sub-154_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-154/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-156/anat/sub-156_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-156/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-157/anat/sub-157_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-157/ses-1/anat 


COPYING /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/fmriprep/sub-159/anat/sub-159_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz to /projects/niblab/bids_projects/Experiments/bbx/bids/derivatives/sub-159/ses-1/anat 


